# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
high_priority = pd.read_csv("csv/high_priority_with_td.csv", index_col=0)
not_high_priority = pd.read_csv("csv/not_high_no_td1.csv", index_col=0)

/tmp/ipykernel_169056/927741048.py:3: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  not_high_priority = pd.read_csv("csv/not_high_no_td1.csv", index_col=0)


In [3]:
contains_stale = not_high_priority['labels'].str.contains("stale", case=False, na=False)
not_high_priority= not_high_priority[~contains_stale].reset_index(drop=True)
not_high_priority.reset_index(drop=True, inplace=True)
not_high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141754e+10,IssuesEvent,2020-02-03 00:00:07,automationbs/testbugreporting,https://api.github.com/repos/automationbs/test...,opened,Default title,bug,default description\n\n|Property | Value|\n|--...
1,1.141754e+10,IssuesEvent,2020-02-03 00:00:12,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,Show alias,Issue-Question Resolution-Answered,"<!--\r\n\r\nFor Windows PowerShell 5.1 issues,..."
2,1.141754e+10,IssuesEvent,2020-02-03 00:00:13,thadiun/hello-world,https://api.github.com/repos/thadiun/hello-world,closed,[Test] testing needs-triage (no delay),triage,NaN
3,1.141754e+10,IssuesEvent,2020-02-03 00:00:16,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,$ENV in using module,Area-Language Issue-Question Resolution-Answered,# Summary of the new feature/enhancement\r\n\r...
4,1.141754e+10,IssuesEvent,2020-02-03 00:00:18,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,Measure-Command does not measure its self,Area-Cmdlets-Utility Issue-Question Resolution...,## The Issue\r\nI'm pretty sure this was just ...
...,...,...,...,...,...,...,...,...,...
9206801,8.109961e+09,IssuesEvent,2018-08-14 09:21:15,urbit/arvo,https://api.github.com/repos/urbit/arvo,opened,%bad-text trips up hall JSON conversion,:hall / :talk cause known marks web interface,"Haven't tested this in detail yet, but pretty ..."
9206802,8.109961e+09,IssuesEvent,2018-08-14 09:21:16,highcharts/highcharts-react,https://api.github.com/repos/highcharts/highch...,closed,HighMaps mapBubble type,pending reply,"Hello,\r\nIt is possible to create also HighMa..."
9206803,8.109982e+09,IssuesEvent,2018-08-14 09:25:07,Loriowar/comindivion,https://api.github.com/repos/Loriowar/comindivion,opened,Add a validation on a belonging of a predicate...,bug data manipulation,Now user can change html content of the intera...
9206804,8.109983e+09,IssuesEvent,2018-08-14 09:25:24,TiiQu-Network/TQ-test-page,https://api.github.com/repos/TiiQu-Network/TQ-...,closed,Error on submit with no values,bug,TypeError: Too few arguments in function sum (...


In [4]:
# Number or different labels
high_priority.labels.value_counts().to_frame()[:50]

,count
labels,
priority: high type: technical debt,31
good first issue high priority preview tech debt 📝documentation/product,19
tech debt high priority preview,16
Priority: High Tech Debt,15
high priority preview tech debt,12
Sprint Priority: High Tech Debt,12
Priority: High Status: Sprint Tech Debt,12
high priority tech debt,10
high priority technical debt,8


In [5]:
high_priority[high_priority["repo"] == "python/mypy"]

,id,type,created_at,repo,repo_url,action,title,labels,body


In [6]:
# Remove mypy from the dataset
high_priority = high_priority[high_priority["repo"] != "python/mypy"]
high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body
0,2.656199e+10,IssuesEvent,2023-01-20 16:36:45,bcgov/cas-cif,https://api.github.com/repos/bcgov/cas-cif,closed,Tech Debt: some prod data must be a migration ...,Tech Debt High Priority,### Description of the Tech Debt ###\r\n\r\nRi...
1,2.227510e+10,IssuesEvent,2022-06-10 15:50:10,department-of-veterans-affairs/va.gov-team,https://api.github.com/repos/department-of-vet...,closed,Remove step from Platform Orientation template,tech-debt Orientation governance-team Ready hi...,"## User Story\nAs a new VFS team member, I wan..."
2,8.882455e+09,IssuesEvent,2019-01-14 13:16:54,GoogleCloudPlatform/google-cloud-eclipse,https://api.github.com/repos/GoogleCloudPlatfo...,opened,Upgrade to latest appengine-plugins-core,Tech Debt high priority ready,The latest release of appengine-plugins-core d...
3,2.330166e+10,IssuesEvent,2022-08-07 12:16:49,KinsonDigital/SWCM,https://api.github.com/repos/KinsonDigital/SWCM,closed,🚧Update issue and PR templates,good first issue high priority preview tech de...,### I have done the items below . . .\n\n- [x]...
4,2.330180e+10,IssuesEvent,2022-08-07 12:39:55,KinsonDigital/GitHubReleaseChecker,https://api.github.com/repos/KinsonDigital/Git...,closed,🚧Update issue and PR templates,good first issue high priority preview tech de...,### I have done the items below . . .\n\n- [x]...
...,...,...,...,...,...,...,...,...,...
679,2.332739e+10,IssuesEvent,2022-08-08 23:04:52,KinsonDigital/GitHubData,https://api.github.com/repos/KinsonDigital/Git...,opened,🚧Fix project item and research issue templates,good first issue high priority tech debt 📝docu...,### Complete The Item Below\n\n- [X] I have up...
680,1.414105e+10,IssuesEvent,2020-11-10 12:07:20,zeebe-io/zeebe,https://api.github.com/repos/zeebe-io/zeebe,closed,Use MAPPED storage level by default,Impact: Performance Impact: Tech Debt Priority...,**Is your feature request related to a problem...
681,2.589332e+10,IssuesEvent,2022-12-14 19:57:19,bcgov/cas-cif,https://api.github.com/repos/bcgov/cas-cif,opened,Tech Debt: some prod data must be a migration ...,Backlog Refinement Tech Debt High Priority,###Description of the Tech Debt###\n\nRight no...
682,1.052572e+10,IssuesEvent,2019-09-30 15:35:16,huridocs/uwazi,https://api.github.com/repos/huridocs/uwazi,closed,Create e2e of custom home-page and custom CSS ...,Priority: High Status: Sprint Tech Debt,This is the first page every client sees. If t...


In [7]:
not_high_priority[not_high_priority["repo"] == "python/mypy"]

,id,type,created_at,repo,repo_url,action,title,labels,body
20492,1.142706e+10,IssuesEvent,2020-02-03 23:33:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Type of conditional expression is object,false-positive needs discussion priority-1-nor...,The type of the following conditional expressi...
20521,1.142707e+10,IssuesEvent,2020-02-03 23:35:24,python/mypy,https://api.github.com/repos/python/mypy,closed,Type inference of Tuples returns object instead,bug false-positive priority-0-high,While interacting with `zip` I encountered a v...
35777,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,feature,"After #5870 is done, it would be nice to also ..."
44233,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,bug,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...
61258,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,feature,**Feature**\r\n\r\nWhen the implementation add...
...,...,...,...,...,...,...,...,...,...
9188054,7.349652e+09,IssuesEvent,2018-03-08 11:27:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Legal Python circular imports result in loss o...,bug priority-1-normal topic-import-cycles,Example:\r\n\r\n`test/__init__.py`:\r\n```pyth...
9188055,7.349652e+09,IssuesEvent,2018-03-08 11:27:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Crash when serializing property with forward r...,crash priority-0-high topic-incremental,This serialization test case fails with a cras...
9188830,7.349972e+09,IssuesEvent,2018-03-08 12:44:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Suggest type annotations for partial types spa...,feature priority-2-low topic-fine-grained-incr...,We don't support inferring partial types from ...
9188843,7.349979e+09,IssuesEvent,2018-03-08 12:45:41,python/mypy,https://api.github.com/repos/python/mypy,closed,What to do about inferring types from multiple...,feature priority-0-high topic-fine-grained-inc...,Typically mypy infers the type of the variable...


In [8]:
# Remove mypy from the dataset
not_high_priority = not_high_priority[not_high_priority["repo"] != "python/mypy"]
not_high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141754e+10,IssuesEvent,2020-02-03 00:00:07,automationbs/testbugreporting,https://api.github.com/repos/automationbs/test...,opened,Default title,bug,default description\n\n|Property | Value|\n|--...
1,1.141754e+10,IssuesEvent,2020-02-03 00:00:12,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,Show alias,Issue-Question Resolution-Answered,"<!--\r\n\r\nFor Windows PowerShell 5.1 issues,..."
2,1.141754e+10,IssuesEvent,2020-02-03 00:00:13,thadiun/hello-world,https://api.github.com/repos/thadiun/hello-world,closed,[Test] testing needs-triage (no delay),triage,NaN
3,1.141754e+10,IssuesEvent,2020-02-03 00:00:16,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,$ENV in using module,Area-Language Issue-Question Resolution-Answered,# Summary of the new feature/enhancement\r\n\r...
4,1.141754e+10,IssuesEvent,2020-02-03 00:00:18,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,Measure-Command does not measure its self,Area-Cmdlets-Utility Issue-Question Resolution...,## The Issue\r\nI'm pretty sure this was just ...
...,...,...,...,...,...,...,...,...,...
9206801,8.109961e+09,IssuesEvent,2018-08-14 09:21:15,urbit/arvo,https://api.github.com/repos/urbit/arvo,opened,%bad-text trips up hall JSON conversion,:hall / :talk cause known marks web interface,"Haven't tested this in detail yet, but pretty ..."
9206802,8.109961e+09,IssuesEvent,2018-08-14 09:21:16,highcharts/highcharts-react,https://api.github.com/repos/highcharts/highch...,closed,HighMaps mapBubble type,pending reply,"Hello,\r\nIt is possible to create also HighMa..."
9206803,8.109982e+09,IssuesEvent,2018-08-14 09:25:07,Loriowar/comindivion,https://api.github.com/repos/Loriowar/comindivion,opened,Add a validation on a belonging of a predicate...,bug data manipulation,Now user can change html content of the intera...
9206804,8.109983e+09,IssuesEvent,2018-08-14 09:25:24,TiiQu-Network/TQ-test-page,https://api.github.com/repos/TiiQu-Network/TQ-...,closed,Error on submit with no values,bug,TypeError: Too few arguments in function sum (...


In [9]:
# Number or different labels
not_high_priority.labels.value_counts().to_frame()[:50]

,count
labels,
enhancement,240489
bug,236887
security vulnerability,52497
question,34353
feature,20431
documentation,17630
Bug,12098
help wanted,10727
wontfix,7180


In [10]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
high_priority["label"] = 1
high_priority["class"] = "high_priority"
high_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,2.656199e+10,IssuesEvent,2023-01-20 16:36:45,bcgov/cas-cif,https://api.github.com/repos/bcgov/cas-cif,closed,Tech Debt: some prod data must be a migration ...,Tech Debt High Priority,### Description of the Tech Debt ###\r\n\r\nRi...,1,high_priority
1,2.227510e+10,IssuesEvent,2022-06-10 15:50:10,department-of-veterans-affairs/va.gov-team,https://api.github.com/repos/department-of-vet...,closed,Remove step from Platform Orientation template,tech-debt Orientation governance-team Ready hi...,"## User Story\nAs a new VFS team member, I wan...",1,high_priority
2,8.882455e+09,IssuesEvent,2019-01-14 13:16:54,GoogleCloudPlatform/google-cloud-eclipse,https://api.github.com/repos/GoogleCloudPlatfo...,opened,Upgrade to latest appengine-plugins-core,Tech Debt high priority ready,The latest release of appengine-plugins-core d...,1,high_priority
3,2.330166e+10,IssuesEvent,2022-08-07 12:16:49,KinsonDigital/SWCM,https://api.github.com/repos/KinsonDigital/SWCM,closed,🚧Update issue and PR templates,good first issue high priority preview tech de...,### I have done the items below . . .\n\n- [x]...,1,high_priority
4,2.330180e+10,IssuesEvent,2022-08-07 12:39:55,KinsonDigital/GitHubReleaseChecker,https://api.github.com/repos/KinsonDigital/Git...,closed,🚧Update issue and PR templates,good first issue high priority preview tech de...,### I have done the items below . . .\n\n- [x]...,1,high_priority


In [11]:
not_high_priority["label"] = 0
not_high_priority["class"] = "not_high_priority" 
not_high_priority.head()

/tmp/ipykernel_169056/996601242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority["label"] = 0
/tmp/ipykernel_169056/996601242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority["class"] = "not_high_priority"


,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.141754e+10,IssuesEvent,2020-02-03 00:00:07,automationbs/testbugreporting,https://api.github.com/repos/automationbs/test...,opened,Default title,bug,default description\n\n|Property | Value|\n|--...,0,not_high_priority
1,1.141754e+10,IssuesEvent,2020-02-03 00:00:12,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,Show alias,Issue-Question Resolution-Answered,"<!--\r\n\r\nFor Windows PowerShell 5.1 issues,...",0,not_high_priority
2,1.141754e+10,IssuesEvent,2020-02-03 00:00:13,thadiun/hello-world,https://api.github.com/repos/thadiun/hello-world,closed,[Test] testing needs-triage (no delay),triage,NaN,0,not_high_priority
3,1.141754e+10,IssuesEvent,2020-02-03 00:00:16,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,$ENV in using module,Area-Language Issue-Question Resolution-Answered,# Summary of the new feature/enhancement\r\n\r...,0,not_high_priority
4,1.141754e+10,IssuesEvent,2020-02-03 00:00:18,PowerShell/PowerShell,https://api.github.com/repos/PowerShell/PowerS...,closed,Measure-Command does not measure its self,Area-Cmdlets-Utility Issue-Question Resolution...,## The Issue\r\nI'm pretty sure this was just ...,0,not_high_priority


In [12]:
# Drop duplicates by the content of the body
high_priority = high_priority.drop_duplicates(subset=['title'], keep='last')
high_priority.dropna(inplace=True)
high_priority.reset_index(inplace=True)
high_priority.drop(columns=["index"] , inplace= True)
high_priority["class"].value_counts()

/tmp/ipykernel_169056/124914565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.dropna(inplace=True)
/tmp/ipykernel_169056/124914565.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.drop(columns=["index"] , inplace= True)


class
high_priority    509
Name: count, dtype: int64

In [13]:
# Drop duplicates by the content of the body
not_high_priority = not_high_priority.drop_duplicates(subset=['title'], keep='last')
not_high_priority.dropna(inplace=True)
not_high_priority.reset_index(inplace=True)
not_high_priority.drop(columns=["index"] , inplace= True)
not_high_priority["class"].value_counts()

/tmp/ipykernel_169056/4269543193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.dropna(inplace=True)
/tmp/ipykernel_169056/4269543193.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.drop(columns=["index"] , inplace= True)


class
not_high_priority    1330864
Name: count, dtype: int64

In [14]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509 entries, 0 to 508
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          509 non-null    float64
 1   type        509 non-null    object 
 2   created_at  509 non-null    object 
 3   repo        509 non-null    object 
 4   repo_url    509 non-null    object 
 5   action      509 non-null    object 
 6   title       509 non-null    object 
 7   labels      509 non-null    object 
 8   body        509 non-null    object 
 9   label       509 non-null    int64  
 10  class       509 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 43.9+ KB


In [15]:
label_counts = high_priority["class"].value_counts()
label_counts_nhp = not_high_priority["class"].value_counts()
print(label_counts)
not_high_priority_count = label_counts_nhp["not_high_priority"]
print(not_high_priority_count)
hp_count = label_counts["high_priority"]
hp_count

class
high_priority    509
Name: count, dtype: int64
1330864


509

In [16]:
not_high_priority = not_high_priority.sample(frac=hp_count/not_high_priority_count, random_state=42)
not_high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
506660,2.774846e+10,IssuesEvent,2023-03-15 18:45:33,MPMG-DCC-UFMG/F01,https://api.github.com/repos/MPMG-DCC-UFMG/F01,closed,Teste de generalizacao para a tag Despesas - P...,generalization test development,DoD: Realizar o teste de Generalização do vali...,0,not_high_priority
1124567,2.608744e+10,IssuesEvent,2022-12-26 06:02:34,orna-memory-hunting/storage,https://api.github.com/repos/orna-memory-hunti...,closed,Increases earthen resistance by 80%,time 00h MSK q.3-Б / в бою выберешь - тайные и...,# Trees & Toxins \r\n### Плюсы\r\n- **Increase...,0,not_high_priority
886278,1.588172e+10,IssuesEvent,2021-04-09 15:08:34,Ultimaker/Cura,https://api.github.com/repos/Ultimaker/Cura,closed,Cura crashes during preview,Status: Duplicate Type: Bug,### Application Version\n\n4.8.0\n\n### Platfo...,0,not_high_priority
180152,1.088138e+10,IssuesEvent,2019-11-17 17:23:15,WarEmu/WarBugs,https://api.github.com/repos/WarEmu/WarBugs,closed,"[Scenario] ""Gates of Ekrund' - Stuck next to Flag",Map/Area Scenarios,I don't know how he got stuck there but if kep...,0,not_high_priority
387370,2.792452e+09,IssuesEvent,2015-05-11 00:39:07,callmecavs/layzr.js,https://api.github.com/repos/callmecavs/layzr.js,closed,Hidden images are loaded,roadmap,"Hey @callmecavs,\r\n\r\nIt appears that images...",0,not_high_priority
...,...,...,...,...,...,...,...,...,...,...,...
1282367,6.203080e+09,IssuesEvent,2017-07-06 10:51:10,CellMigStandOrg/biotracks,https://api.github.com/repos/CellMigStandOrg/b...,opened,Unify standard names structure,enhancement,See https://github.com/CellMigStandOrg/biotrac...,0,not_high_priority
693907,2.277476e+10,IssuesEvent,2022-07-08 13:29:58,steinbergmedia/vstgui,https://api.github.com/repos/steinbergmedia/vs...,closed,Mouse Move may have wrong coordinates,bug,"When no mouse button is pressed, the mouse loc...",0,not_high_priority
824447,7.460640e+09,IssuesEvent,2018-03-30 20:39:51,distillpub/template,https://api.github.com/repos/distillpub/template,closed,Using colons in title field of frontmatter,enhancement,Using a colon in the title field (or other fie...,0,not_high_priority
1307183,6.890378e+09,IssuesEvent,2017-11-22 13:49:58,ExpressGateway/express-gateway,https://api.github.com/repos/ExpressGateway/ex...,closed,Config file watcher on network mount won't rec...,bug,When running in Docker with the config on a mo...,0,not_high_priority


In [17]:
all_priority = pd.concat([high_priority,not_high_priority] , ignore_index = True)
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
1013,6.203080e+09,IssuesEvent,2017-07-06 10:51:10,CellMigStandOrg/biotracks,https://api.github.com/repos/CellMigStandOrg/b...,opened,Unify standard names structure,enhancement,See https://github.com/CellMigStandOrg/biotrac...,0,not_high_priority
1014,2.277476e+10,IssuesEvent,2022-07-08 13:29:58,steinbergmedia/vstgui,https://api.github.com/repos/steinbergmedia/vs...,closed,Mouse Move may have wrong coordinates,bug,"When no mouse button is pressed, the mouse loc...",0,not_high_priority
1015,7.460640e+09,IssuesEvent,2018-03-30 20:39:51,distillpub/template,https://api.github.com/repos/distillpub/template,closed,Using colons in title field of frontmatter,enhancement,Using a colon in the title field (or other fie...,0,not_high_priority
1016,6.890378e+09,IssuesEvent,2017-11-22 13:49:58,ExpressGateway/express-gateway,https://api.github.com/repos/ExpressGateway/ex...,closed,Config file watcher on network mount won't rec...,bug,When running in Docker with the config on a mo...,0,not_high_priority
1017,2.204698e+10,IssuesEvent,2022-05-30 03:38:51,madhans23/linux-4.1.15,https://api.github.com/repos/madhans23/linux-4...,closed,CVE-2020-16119 (High) detected in linux-stable...,security vulnerability,## CVE-2020-16119 - High Severity Vulnerabilit...,0,not_high_priority


In [18]:
# Does this need to be more balanced?
all_priority["label"].value_counts()

label
1    509
0    509
Name: count, dtype: int64

In [19]:
print(all_priority["title"][0])
print(all_priority["body"][0])

Remove step from Platform Orientation template
## User Story
As a new VFS team member, I want to not encounter any unobtainable steps in Platform Orientation so that I can easily complete the steps in the Platform Orientation process.

- Intended Audience: VFS teams
- Desired Outcome: Platform Orientation gh template updated
- Due Date: TBD

## Description
In ticket #39321, Governance team added the process of getting added to vets.gov-write GH team to the Platform Orientation process, as opposed to a step that occurs before opening this ticket. There are certain steps that cannot be completed now that vets.gov-write access is now part of this process, including adding GH labels to the ticket. As such, the step, " Add your teams github label under Labels" needs to be removed from the ticket.

## Artifacts Needing Updates
[Platform Orientation Template](https://github.com/department-of-veterans-affairs/va.gov-team/blob/master/.github/ISSUE_TEMPLATE/orientation-epic.md)

## Tasks
- [x] R

In [20]:
# Copy content of body to a new col named text
all_priority["text"] = all_priority["title"] + " " + all_priority["body"]
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class,text
1013,6.203080e+09,IssuesEvent,2017-07-06 10:51:10,CellMigStandOrg/biotracks,https://api.github.com/repos/CellMigStandOrg/b...,opened,Unify standard names structure,enhancement,See https://github.com/CellMigStandOrg/biotrac...,0,not_high_priority,Unify standard names structure See https://git...
1014,2.277476e+10,IssuesEvent,2022-07-08 13:29:58,steinbergmedia/vstgui,https://api.github.com/repos/steinbergmedia/vs...,closed,Mouse Move may have wrong coordinates,bug,"When no mouse button is pressed, the mouse loc...",0,not_high_priority,Mouse Move may have wrong coordinates When no ...
1015,7.460640e+09,IssuesEvent,2018-03-30 20:39:51,distillpub/template,https://api.github.com/repos/distillpub/template,closed,Using colons in title field of frontmatter,enhancement,Using a colon in the title field (or other fie...,0,not_high_priority,Using colons in title field of frontmatter Usi...
1016,6.890378e+09,IssuesEvent,2017-11-22 13:49:58,ExpressGateway/express-gateway,https://api.github.com/repos/ExpressGateway/ex...,closed,Config file watcher on network mount won't rec...,bug,When running in Docker with the config on a mo...,0,not_high_priority,Config file watcher on network mount won't rec...
1017,2.204698e+10,IssuesEvent,2022-05-30 03:38:51,madhans23/linux-4.1.15,https://api.github.com/repos/madhans23/linux-4...,closed,CVE-2020-16119 (High) detected in linux-stable...,security vulnerability,## CVE-2020-16119 - High Severity Vulnerabilit...,0,not_high_priority,CVE-2020-16119 (High) detected in linux-stable...


In [21]:
all_priority["text"][0]

'Remove step from Platform Orientation template ## User Story\nAs a new VFS team member, I want to not encounter any unobtainable steps in Platform Orientation so that I can easily complete the steps in the Platform Orientation process.\n\n- Intended Audience: VFS teams\n- Desired Outcome: Platform Orientation gh template updated\n- Due Date: TBD\n\n## Description\nIn ticket #39321, Governance team added the process of getting added to vets.gov-write GH team to the Platform Orientation process, as opposed to a step that occurs before opening this ticket. There are certain steps that cannot be completed now that vets.gov-write access is now part of this process, including adding GH labels to the ticket. As such, the step, " Add your teams github label under Labels" needs to be removed from the ticket.\n\n## Artifacts Needing Updates\n[Platform Orientation Template](https://github.com/department-of-veterans-affairs/va.gov-team/blob/master/.github/ISSUE_TEMPLATE/orientation-epic.md)\n\n##

In [22]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "label" , "class"]]
all_priority_subset

,text,label,class
0,Remove step from Platform Orientation template...,1,high_priority
1,Upgrade to latest appengine-plugins-core The l...,1,high_priority
2,Implement file system based throttler Will nip...,1,high_priority
3,Finish Search v2 queryString custom extension ...,1,high_priority
4,Update versions & fix all audit warnings Last ...,1,high_priority
...,...,...,...
1013,Unify standard names structure See https://git...,0,not_high_priority
1014,Mouse Move may have wrong coordinates When no ...,0,not_high_priority
1015,Using colons in title field of frontmatter Usi...,0,not_high_priority
1016,Config file watcher on network mount won't rec...,0,not_high_priority


In [23]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_169056/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [24]:
all_priority_subset

,text,label,class,text_str
0,Remove step from Platform Orientation template...,1,high_priority,Remove step from Platform Orientation template...
1,Upgrade to latest appengine-plugins-core The l...,1,high_priority,Upgrade to latest appengine-plugins-core The l...
2,Implement file system based throttler Will nip...,1,high_priority,Implement file system based throttler Will nip...
3,Finish Search v2 queryString custom extension ...,1,high_priority,Finish Search v2 queryString custom extension ...
4,Update versions & fix all audit warnings Last ...,1,high_priority,Update versions & fix all audit warnings Last ...
...,...,...,...,...
1013,Unify standard names structure See https://git...,0,not_high_priority,Unify standard names structure See https://git...
1014,Mouse Move may have wrong coordinates When no ...,0,not_high_priority,Mouse Move may have wrong coordinates When no ...
1015,Using colons in title field of frontmatter Usi...,0,not_high_priority,Using colons in title field of frontmatter Usi...
1016,Config file watcher on network mount won't rec...,0,not_high_priority,Config file watcher on network mount won't rec...


In [25]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_169056/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [26]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,remove step from platform orientation template...,1
1,upgrade to latest appenginepluginscore the lat...,1
2,implement file system based throttler will nip...,1
3,finish search querystring custom extension to ...,1
4,update versions fix all audit warnings last ve...,1
...,...,...
1013,unify standard names structure see,0
1014,mouse move may have wrong coordinates when no ...,0
1015,using colons in title field of frontmatter usi...,0
1016,config file watcher on network mount wont reco...,0


In [27]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_169056/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_169056/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,label
0,remove step from platform orientation template...,1
1,upgrade to latest appenginepluginscore the lat...,1
2,implement file system based throttler will nip...,1
3,finish search querystring custom extension to ...,1
4,update versions fix all audit warnings last ve...,1
...,...,...
955,unify standard names structure see,0
956,mouse move may have wrong coordinates when no ...,0
957,using colons in title field of frontmatter usi...,0
958,config file watcher on network mount wont reco...,0


In [28]:
# Print all text_clean rows in the dataframe
for i in range(0, len(priority_label_text)):
    print(priority_label_text["text_clean"][i])

remove step from platform orientation template user storyas a new vfs team member i want to not encounter any unobtainable steps in platform orientation so that i can easily complete the steps in the platform orientation process intended audience vfs teams desired outcome platform orientation gh template updated due date tbd descriptionin ticket governance team added the process of getting added to vetsgovwrite gh team to the platform orientation process as opposed to a step that occurs before opening this ticket there are certain steps that cannot be completed now that vetsgovwrite access is now part of this process including adding gh labels to the ticket as such the step add your teams github label under labels needs to be removed from the ticket artifacts needing updates tasks remove how to fill out this ticket add your teams github label under labels review other steps to determine if any other steps need removed acceptance criteria github template platform orientation has been up

In [29]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
priority_label_text.to_csv("csv/clean_priority_debt_high_or_not_high_v2.csv")

In [30]:
pri = pd.read_csv("csv/clean_priority_debt_high_or_not_high_v2.csv", index_col=0)
pri

,text_clean,label
0,remove step from platform orientation template...,1
1,upgrade to latest appenginepluginscore the lat...,1
2,implement file system based throttler will nip...,1
3,finish search querystring custom extension to ...,1
4,update versions fix all audit warnings last ve...,1
...,...,...
955,unify standard names structure see,0
956,mouse move may have wrong coordinates when no ...,0
957,using colons in title field of frontmatter usi...,0
958,config file watcher on network mount wont reco...,0


In [31]:
# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test


In [32]:
train , validate , test = train_validate_test_split(priority_label_text)

In [33]:
from datasets import Dataset, DatasetDict
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 96
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 768
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 96
    })
})

In [34]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
from datasets import Dataset

# Convert the DataFrame
hf_dataset = Dataset.from_pandas(priority_label_text)
hf_dataset

Dataset({
    features: ['text_clean', 'label'],
    num_rows: 960
})

In [37]:
ds.push_to_hub("high_priority_or_not_high_2")


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]